In [1]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
tablemap={"capital_emr_etl":"capital"} #,
first=" select 'insert' as kind, '2021-01-01 23:59:59' as etldate,0 as  etlindex, "
end=" , '2021' as year,   '01' as month,  '01' as day from  "
def getTableColum(tableNm,b):
    colum=""
    for index in range(len(b)):
        colum=colum+setDef(b[index]["data_type"],b[index]["col_name"],tableNm)+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col,tableNm):
    if(type[:7] == 'boolean' ) :
        return "cast ("+table_col+" as string ) as "+table_col
#     elif tableNm=='t_package_details' and  type == 'timestamp' and (table_col=='start_time' or table_col=='end_time') :
#         return "substring ("+table_col+",11,9 ) as "+table_col
    else:
        return table_col
def make_col_temptable(databaseName,row,stream_tableList):
    tableName=row["tableName"]
    print(tableName)
#     if tableName in stream_tableList and (tableName=='t_thirdparty_data' or tableName=='t_clear_detail_log') :
    if  tableName=='t_case_distribution_log' :
        tablecolum="desc " +databaseName+"."+tableName;
        tableSchema= spark.sql(tablecolum).collect()
        sql=first+ getTableColum(tableName,tableSchema)+end+databaseName+"."+tableName
        path="s3://rupiahplus-data-warehouse/stream/"+tablemap[databaseName]+"_etl/"+tableName+"/"
        spark.sql(sql).write.mode("append").partitionBy("year","month","day").orc(path)
if __name__ == "builtins":
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    df=spark.sql("""show tables in banda_stream_etl""").collect()
    stream_tableList=[]
    for i in df:
        stream_tableList.append(i['tableName'])
    for databaseName in tablemap:
        databasesql="show tables in "+"capital_emr_etl"
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm,stream_tableList) for tableNm in tablelist])
            for  future in futures_result[0]:
                print(future.result())


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1623318899558_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-1:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



t_capital_account
t_capital_account_record
t_debt_record
t_interest_record
t_mq_listener_original
t_order_original
t_package
t_package_details
t_package_details_record
t_repay_record
t_settlement_record
t_subject
t_workday_schedule
None
None
None
None
None
None
None
None
None
None
None
None
None

In [1]:
spark.sql("""
select  substring(start_time,) as start_time, substring(end_time,11,9) as end_time ,* from capital_emr_etl.t_package_details

""").printSchema()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1611540126777_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- id: long (nullable = true)
 |-- package_id: long (nullable = true)
 |-- capital_account_record_id: long (nullable = true)
 |-- charge_amount: decimal(18,2) (nullable = true)
 |-- available_amount: decimal(18,2) (nullable = true)
 |-- subject_type: string (nullable = true)
 |-- match_priority: short (nullable = true)
 |-- opened: short (nullable = true)
 |-- workday: short (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- buy_back_rule: string (nullable = true)
 |-- day_rate: decimal(16,6) (nullable = true)
 |-- create_time: timestamp (nullable = true)
 |-- update_time: timestamp (nullable = true)
 |-- limit_amount: decimal(18,2) (nullable = true)
 |-- swap_package_details_id: long (nullable = true)
 |-- user_type: short (nullable = true)
 |-- interest_type: string (nullable = true)